In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.metrics.pairwise import cosine_similarity


from sklearn.feature_extraction.text import TfidfVectorizer
# Any results you write to the current directory are saved as output.  

/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/movie.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/link.csv


In [2]:
os.listdir('/kaggle/input/movielens-20m-dataset/')

['tag.csv',
 'rating.csv',
 'genome_scores.csv',
 'movie.csv',
 'genome_tags.csv',
 'link.csv']

In [3]:
import pandas as pd

In [4]:
train_csv = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')

In [5]:
train_csv = train_csv.drop(columns = ['timestamp'],axis = 1)

In [6]:
train_csv_full = train_csv

In [7]:
len(train_csv)

20000263

In [8]:
users  = train_csv.userId.unique()
print(len(users))

138493


In [9]:
items = train_csv.movieId.unique()
len(items)

26744

In [10]:
userid2idx = {o:i+1 for i,o in enumerate(users)}
movieid2idx = {o:i+1 for i,o in enumerate(items)}

In [11]:
len(movieid2idx)

26744

In [12]:
movie_csv = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')

In [13]:
#print(movieid2idx)

In [14]:
#print(userid2idx)

In [15]:
df = pd.DataFrame(columns = ['userid','movieid','rating'])

In [16]:
df['userid'] = train_csv.userId.apply(lambda x: userid2idx[x])
df['movieid'] = train_csv.movieId.apply(lambda x: movieid2idx[x])
df['rating'] = 1

In [17]:
num_users = train_csv['userId'].nunique()
print(num_users)

138493


In [18]:
num_items = train_csv['movieId'].nunique()
print(num_items)

26744


COLLABORATIVE FILTERING

In [19]:
from keras.layers import Embedding, Input, Flatten,Dense
from keras.models import Model
from keras.layers.merge import concatenate,multiply


Using TensorFlow backend.


In [20]:
user_input = Input(shape = (1,))
item_input = Input(shape = (1,))

In [21]:
latent_factor = 64
layers = [512,256,128,64]
mf_embedding_user = Embedding(input_dim = num_users,output_dim = latent_factor,name = 'mf_e')(user_input)
mf_embedding_item = Embedding(input_dim = num_items,output_dim = latent_factor,name = 'mf_i')(item_input)
o = multiply([mf_embedding_user,mf_embedding_item],name= 'combine_mf')
o = Flatten()(o)

mlp_embedding_user = Embedding(input_dim = num_users,output_dim = layers[0],name = 'mlp_e')(user_input)
mlp_embedding_item = Embedding(input_dim = num_items,output_dim = layers[0],name = 'mlp_i')(item_input)
mlp_vector = concatenate([mlp_embedding_user,mlp_embedding_item])
for i in range(1,len(layers)):
    layer = Dense(layers[i])
    mlp_vector = layer(mlp_vector)
mlp_vector = Flatten()(mlp_vector)
c= concatenate([o,mlp_vector])

c = Dense(1,activation= 'sigmoid')(c)
model = Model(inputs = [user_input, item_input], outputs = c)

# Minimize binary cross entropy
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
mlp_e (Embedding)               (None, 1, 512)       70908416    input_1[0][0]                    
__________________________________________________________________________________________________
mlp_i (Embedding)               (None, 1, 512)       13692928    input_2[0][0]                    
____________________________________________________________________________________________

In [23]:
train_csv.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [24]:

def training_dataset(df,num_neg):
    labels = []
    user_input = []
    item_input = []
    for i in range(0,len(df)):
        user_input.append(df['userid'][i])
        item_input.append(df['movieid'][i])
        labels.append(1)
        for j in range(0,num_neg):
            k = np.random.randint(num_items)
            user_input.append(df['userid'][i])
            item_input.append(k)
            labels.append(0)
        
        
        
    return user_input,item_input,labels
        

In [25]:
num_neg = 1
user_train,item_train,labels = training_dataset(df[0:200000],num_neg)

In [26]:
len(user_train)

400000

In [27]:
 hist = model.fit([np.array(user_train), np.array(item_train)], #input
                         np.array(labels), epochs=1,validation_split = 0.1)

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 357s 991us/step - loss: 0.3411 - accuracy: 0.8664 - val_loss: 0.2833 - val_accuracy: 0.8886


In [28]:
hist.history['loss']

[0.3411068756788969]

**FINDING SIMILAR USERS **

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
arr = model.predict([np.array(user_train[0:10000]),np.array(item_train[0:10000])])

In [32]:
arr

array([[0.95938015],
       [0.8840406 ],
       [0.91655916],
       ...,
       [0.12139067],
       [0.98942995],
       [0.08315036]], dtype=float32)

In [33]:
from collections import defaultdict
movieid2idx
removieid2idx = defaultdict(int)
for key,value in movieid2idx.items():
    removieid2idx[value] = key
#print(removieid2idx)

In [34]:
keys = {}
for i,j in movie_csv[['movieId','title']].values:
    keys[i] = j
    

In [35]:
i =0
##MOVIEID2iDX TO IDENTIFY THE MOVIE NAMES
while(user_train[i]==1):
    i = i+1
for j in range(0,i):
    print("users: {}".format(user_train[j]))
    print("items: {}".format(item_train[j]))
#     print("movie id: {}".format(removieid2idx[item_train[j]]))
    x = removieid2idx[item_train[j]]
    print("movie name: {}".format(keys[x]))
    

users: 1
items: 1
movie name: Jumanji (1995)
users: 1
items: 1369
movie name: Risky Business (1983)
users: 1
items: 2
movie name: City of Lost Children, The (Cité des enfants perdus, La) (1995)
users: 1
items: 16465
movie name: Borderland (2007)
users: 1
items: 3
movie name: Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
users: 1
items: 23061
movie name: Eila, Rampe and Likka (2014)
users: 1
items: 4
movie name: Seven (a.k.a. Se7en) (1995)
users: 1
items: 233
movie name: Species (1995)
users: 1
items: 5
movie name: Usual Suspects, The (1995)
users: 1
items: 24907
movie name: Warrendale (1967)
users: 1
items: 6
movie name: Rumble in the Bronx (Hont faan kui) (1995)
users: 1
items: 11869
movie name: Fly Me to the Moon (2008)
users: 1
items: 7
movie name: Rob Roy (1995)
users: 1
items: 1122
movie name: Remember the Titans (2000)
users: 1
items: 8
movie name: Clerks (1994)
users: 1
items: 19539
movie name: King of the Jungle (2000)
users: 1
items: 9
movie name: Interview with the Vampire: The V

CONTENT BASED FILTERING 

In [36]:
movie_csv = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')

In [37]:
tags_csv= pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv')

In [38]:
tags_csv.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [39]:
from collections import defaultdict
movieid = defaultdict(list)
for i in range(0,len(tags_csv)):
    movieid[tags_csv['movieId'].iloc[i]].append(str(tags_csv['tag'].iloc[i]))
        

In [40]:
len(movieid)

19545

In [41]:
movieidslist = []
moviemap = defaultdict()
j = 0
for key,values in movieid.items():
    movieidslist.append((' ').join(values))
    moviemap[key] = j
    j =j+1
    

In [42]:
moviemap
moviemapreverse = defaultdict(list)
for key,value in moviemap.items():
    moviemapreverse[value] = key

In [43]:
#print(moviemapreverse)


In [44]:
#print(moviemap)


In [45]:
vectorizer = TfidfVectorizer(max_features  = 3000)
vectors = vectorizer.fit_transform(movieidslist)

In [46]:
# vectors.shape
# vectors.shape
array = vectors.toarray()

In [47]:
array.shape

(19545, 3000)

EXTRACTING FEATURES USING AUTOENCODERS

In [48]:
from keras import models
from keras.models import Model

from keras.layers import Dense,Embedding,Input,Flatten,Dropout
from keras.optimizers import Adam

In [49]:
def getmodel(latent_factors,features):
    model = models.Sequential()
    model.add(Dense(4096,activation = 'relu', input_shape=(24111,)))
    model.add(Dense(1024,activation = 'relu'))
    model.add(Dense(512,activation= 'relu'))
    model.add(Dense(128,activation = 'relu'))
    model.add(Dense(latent_factors,activation = 'relu'))
    model.add(Dense(128,activation = 'relu'))
    model.add(Dense(256,activation = 'relu'))
    model.add(Dense(1024,activation = 'relu'))
    model.add(Dense(4096,activation = 'relu'))
    model.add(Dense(features,activation = 'sigmoid'))
    return model
                    
def getmodel1(latent_factors, features):
    inp1 = Input(shape=(features,))
    #d1 = Dense(4096, activation='relu')(inp1)
    #d1 = Dropout(0.3)(d1)
    #d1 = Dense(1024, activation='relu')(inp1)
    #d1 = Dense(512, activation='relu')(inp1)
    #d1 = Dense(128, activation='relu')(d1)
    d1 = Dense(latent_factors, activation='relu')(inp1)
   #d1 = Dense(128, activation='relu')(d1)
    #d1 = Dense(512, activation='tanh')(d1)
    #d1 = Dense(1024, activation='relu')(d1)
    #|d1 = Dense(4096, activation='relu')(d1)
    d1 = Dense(features, activation='sigmoid')(d1)
    model = Model(inputs=inp1, outputs=d1)
    
    return model

In [50]:
print(vectors.shape)
features = vectors.shape[1]
#print(features.shape)
#features = features.reshape(-1,)
latentfactors = 32
model = getmodel1(latentfactors,features)
model.compile(loss = 'binary_crossentropy',optimizer = Adam(0.01),metrics = ['acc'])
model.summary()

(19545, 3000)
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 3000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                96032     
_________________________________________________________________
dense_6 (Dense)              (None, 3000)              99000     
Total params: 195,032
Trainable params: 195,032
Non-trainable params: 0
_________________________________________________________________


In [51]:
model_autoencoder = model.fit(array,array,epochs = 1,validation_split = 0.1)

Train on 17590 samples, validate on 1955 samples
Epoch 1/1
17590/17590 [==============================] - 2s 128us/step - loss: 0.0272 - acc: 0.9944 - val_loss: 0.0041 - val_acc: 0.9991


In [52]:
model.save('/kaggle/working/model_autoencoder.h5')

len(model.layers)

In [53]:
model.layers

In [54]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 3000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                96032     
_________________________________________________________________
dense_6 (Dense)              (None, 3000)              99000     
Total params: 195,032
Trainable params: 195,032
Non-trainable params: 0
_________________________________________________________________


In [55]:
len(model.layers)

3

In [56]:
modell = models.Sequential()
count = 0
for i in model.layers:
    if(count<2):
        modell.add(i)
    count = count+1

In [57]:
modell.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                96032     
Total params: 96,032
Trainable params: 96,032
Non-trainable params: 0
_________________________________________________________________


In [58]:
def getid(x):
    return moviemapreverse[x]
def getidreverse(x):
    return moviemap[x]

CHECK PREDICTIONS OF PREVIOUS ITEMS

In [59]:
def getinfo(x):
    iid = getid(x)
    moviename = np.where(movie_csv['movieId'] == iid)
    moviename = list(moviename)[0]
    moviename = moviename[0]
    print("movie_name :{}".format(movie_csv.iloc[moviename]))
    print('\n Movie descriptions: {}'.format(movieidslist[x]))


In [60]:
a = []
a.append(movieidslist[875])
#getinfo(875)
vector_item = vectorizer.transform(a)
ans = modell.predict(vector_item)

In [61]:
b = []
b.append(movieidslist[87])
#getinfo(87)
vector_items1 = vectorizer.transform(b)
ansa = modell.predict(vector_items1)

In [62]:
val = cosine_similarity(ans,ansa)

In [63]:
print(val)

[[0.99976265]]


In [64]:
len(movieidslist)
predictlistvectors= []

for i in movieidslist:
    x = []
    x.append(i)
    y = vectorizer.transform(x)
    predictlistvectors.append(modell.predict(y))

In [65]:
#START FROM HERE
#ITERATE THROUGH LIST OF ITEMS AND WITH THE USE OF EMBEDDINGS FIND THE MOST SIMILAR ITEM USING COSINE SIMILARITY 
#ITEM 1 PREDICTIONS ARE FOUND. CREATE DEFINE FOR GETTING INFORMATIONS
# predictions = []
# for i,j in enumerate(movieidslist):
    
    

In [66]:
cosines = []
for i in predictlistvectors:
    sim = cosine_similarity(ans,i)
    cosines.append(sim)

In [67]:
from collections import defaultdict
cosines_id = []
for i,j in enumerate(cosines):
    cosines_id.append([i,j])

In [68]:
def keysort(cosine_id):
    return cosine_id[1]
    

In [69]:
cosines_id.sort(reverse= True,key = keysort)

**PREDICTIONS SUMMARY**[](http://)

In [70]:
for i in cosines_id[0:10]:
    getinfo(i[0])

movie_name :movieId                                                 1196
title      Star Wars: Episode V - The Empire Strikes Back...
genres                               Action|Adventure|Sci-Fi
Name: 1171, dtype: object

 Movie descriptions: action adventure classic fantasy George Lucas great soundtrack Harrison Ford music original plot sci-fi sequel star wars sword fight sword fighting adventure fantasy father-son relationship Harrison Ford space dvd action adventure classic George Lucas great soundtrack Harrison Ford space star wars sci-fi space classic sci-fi star wars fantasy war androids Carrie Fisher fantasy George Lucas Harrison Ford Industrial Light and Magic Irvin Kershner John Williams Lawrence Kasdan Mark Hamill robots Space adventure boba fett darth vader fantasy father-son relationship George Lucas Harrison Ford music robots space Space battle sword fight adventure fantasy father-son relationship Harrison Ford sci-fi space star wars Futuristmovies.com George Lucas Harriso